In [2]:
!pip3 install pg8000==1.31.2 boto3==1.34.136 uuid==1.30

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 1.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6503 sha256=1625c355eca753a1b4313ce9547236a370816b9e7eb7adca0dd028d3765fe7be
  Stored in directory: /root/.cache/pip/wheels/ed/08/9e/f0a977dfe55051a07e21af89200125d65f1efa60cbac61ed88
Successfully built uuid


In [1]:
import os
import boto3
import dotenv
from libs.utils.connector_llm import *

dotenv.load_dotenv()

True

In [23]:
llm = factory_create_connector_llm(
    provider='bedrock',
    modelname='anthropic.claude-3-5-sonnet-20240620-v1:0',
    credentials=CredentialsBedrock(
        aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.getenv("AWS_ACCESS_KEY_SECRET"),
    ),
)

# chat completion, normal
chat_completion = llm.chat_completion([
    ChatCompletionMessage(role='system', content='You must respond to the user in a mocking way, making puns'),
    ChatCompletionMessage(role='user', content='hi, my name is leonardo'),
])
assert type(chat_completion.choices[0].message.content) == str
assert len(chat_completion.choices[0].message.content) > 2
print(chat_completion.choices[0].message.content)

Well, well, well, if it isn't Leonardo! I hope you're not too 'shell-shocked' by my response. Are you here to 'turtle-y' dominate the conversation, or are you just feeling a bit 'green' around the gills? Don't worry, I won't 'ninja' your time - I'm just here to have some fun! So, what's on your mind, oh great Renaissance master? Painting me a picture of your day, perhaps?


In [24]:
# chat completion, stream
chat_completion_stream = llm.chat_completion_stream([
    ChatCompletionMessage(role='system', content='You must respond to the user in a mocking way, making puns'),
    ChatCompletionMessage(role='user', content='hi, my name is leonardo'),
])

answer = ''
for chunk in chat_completion_stream:
    if len(chunk.choices) > 0:
        message = chunk.choices[0].message
        if message.content:
            print(message.content, end='')
            answer += message.content
            assert type(message.content) == str
assert len(answer) > 2

Well, well, well, if it isn't Leo-nardo! I hope you're not feeling too blue about your name, though I'm sure it's been quite the masterpiece of conversation over the years. Did you bring your turtle friends along, or are you flying solo on this chat adventure? Either way, I'm sure we'll have a shell of a time!